<a href="https://colab.research.google.com/github/kili-technology/kili-python-sdk/blob/master/recipes/label_parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to use the label parser

In [ ]:
%pip install kili

In [ ]:
from kili.client import Kili
from kili.utils.labels.parsing import ParsedLabel

In [ ]:
kili = Kili()

## ParsedLabel class

The `ParsedLabel` class represents a Kili label.

This class directly inherits from `dict`, and thus behaves like a dictionary.

In [ ]:
print(ParsedLabel.__bases__[0])

<class 'dict'>


Converting a label to a `ParsedLabel` is as simple as:

In [ ]:
json_interface = {
    "jobs": {
        "CLASSIFICATION_JOB": {
            "content": {
                "categories": {
                    "A": {"children": [], "name": "A"},
                    "B": {"children": [], "name": "B"},
                },
                "input": "radio",
            },
            "instruction": "Class",
            "mlTask": "CLASSIFICATION",
            "required": 1,
            "isChild": False,
        }
    }
}

In [ ]:
my_label = {
    "author": {"email": "first.last@kili-technology.com", "id": "123456"},
    "id": "clh0fsi9u0tli0j666l4sfhpz",
    "jsonResponse": {"CLASSIFICATION_JOB": {"categories": [{"confidence": 100, "name": "A"}]}},
    "labelType": "DEFAULT",
    "secondsToLabel": 5,
}

my_parsed_label = ParsedLabel(my_label, json_interface=json_interface, input_type="IMAGE")

In [ ]:
print(my_parsed_label["author"]["email"])

first.last@kili-technology.com


The `jsonResponse` dict key is not accessible anymore:

In [ ]:
try:
    my_parsed_label["jsonResponse"]
except KeyError as err:
    print(f"The key {err} is not accessible anymore.")

The key 'jsonResponse' is not accessible anymore.


It is replaced with the `.jobs` attribute instead.

## .jobs attribute

The `.jobs` attribute of a `ParsedLabel` class is a dictionary-like object that contains the parsed labels.

The keys are the names of the jobs, and the values are the parsed job responses.

Let's create a simple Kili project to illustrate this.

## Classification job

We define a json interface for a two classification jobs:

- a single-class classification job, with name `SINGLE_CLASS_JOB` and three categories `A`, `B` and `C`
- a multi-class classification job, with name `MULTI_CLASS_JOB` and three categories `D`, `E` and `F`.

In [ ]:
json_interface = {
    "jobs": {
        "SINGLE_CLASS_JOB": {
            "content": {
                "categories": {
                    "A": {"children": [], "name": "A"},
                    "B": {"children": [], "name": "B"},
                    "C": {"children": [], "name": "C"},
                },
                "input": "radio",
            },
            "instruction": "Class",
            "mlTask": "CLASSIFICATION",
            "required": 1,
            "isChild": False,
        },
        "MULTI_CLASS_JOB": {
            "content": {
                "categories": {
                    "D": {"children": [], "name": "D"},
                    "E": {"children": [], "name": "E"},
                    "F": {"children": [], "name": "F"},
                },
                "input": "checkbox",
            },
            "instruction": "Class",
            "mlTask": "CLASSIFICATION",
            "required": 1,
            "isChild": False,
        },
    }
}
project_id = kili.create_project(
    input_type="TEXT", json_interface=json_interface, title="Label parsing tutorial"
)["id"]

We also upload some assets to the project:

In [ ]:
kili.append_many_to_dataset(
    project_id,
    content_array=["text1", "text2", "text3"],
    external_id_array=["asset1", "asset2", "asset3"],
);

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  1.52it/s]


Once the assets are uploaded, we can start labeling them manually through the Kili UI.

For this tutorial, we will just upload already existing labels.

In [ ]:
labels_to_upload = [
    {
        "SINGLE_CLASS_JOB": {"categories": [{"confidence": 75, "name": "A"}]},
        "MULTI_CLASS_JOB": {
            "categories": [{"confidence": 1, "name": "D"}, {"confidence": 1, "name": "E"}]
        },
    },
    {
        "SINGLE_CLASS_JOB": {"categories": [{"confidence": 50, "name": "B"}]},
        "MULTI_CLASS_JOB": {
            "categories": [{"confidence": 2, "name": "E"}, {"confidence": 2, "name": "F"}]
        },
    },
    {
        "SINGLE_CLASS_JOB": {"categories": [{"confidence": 25, "name": "C"}]},
        "MULTI_CLASS_JOB": {
            "categories": [{"confidence": 3, "name": "F"}, {"confidence": 3, "name": "D"}]
        },
    },
]
kili.append_labels(
    json_response_array=labels_to_upload,
    project_id=project_id,
    asset_external_id_array=["asset1", "asset2", "asset3"],
);

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.95it/s]


When querying labels using `kili.labels()`, it is possible to automatically parse the labels using the `output_format` argument:

In [ ]:
# labels is a list of ParsedLabel object
labels = kili.labels(project_id, output_format="parsed_label")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 11.86it/s]


In [ ]:
print(len(labels))

3


In [ ]:
print(type(labels[0]))

<class 'kili.utils.labels.parsing.ParsedLabel'>


Using the `.jobs` attribute with the job name, one can access the label's data:

In [ ]:
print(labels[0].jobs["SINGLE_CLASS_JOB"])

{'categories': [{'name': 'A', 'confidence': 75}]}


In [ ]:
print(labels[0].jobs["SINGLE_CLASS_JOB"].categories)

[{'name': 'A', 'confidence': 75}]


In [ ]:
print(labels[0].jobs["SINGLE_CLASS_JOB"].categories[0].name)

A


Since `SINGLE_CLASS_JOB` is a single-category classification job, the `.category` attribute is available, and is an alias for `.categories[0]`:

In [ ]:
print(labels[0].jobs["SINGLE_CLASS_JOB"].category.name)
print(labels[0].jobs["SINGLE_CLASS_JOB"].category.confidence)

A
75


The `.category` attribute is forbidden for multi-categories classification jobs:

In [ ]:
try:
    print(labels[0].jobs["MULTI_CLASS_JOB"].category.name)
except Exception as err:
    print("Error: ", err)

Error:  The attribute 'category' is not compatible with the job.


It is also possible to iterate over the job names:

In [ ]:
for i, label in enumerate(labels):
    print(f"\nLabel {i}")
    for job_name, job_data in label.jobs.items():
        print("job_name: ", job_name)
        for category in job_data.categories:
            print("category: ", category.name, category.confidence)


Label 0
job_name:  SINGLE_CLASS_JOB
category:  A 75
job_name:  MULTI_CLASS_JOB
category:  D 1
category:  E 1

Label 1
job_name:  SINGLE_CLASS_JOB
category:  B 50
job_name:  MULTI_CLASS_JOB
category:  E 2
category:  F 2

Label 2
job_name:  SINGLE_CLASS_JOB
category:  C 25
job_name:  MULTI_CLASS_JOB
category:  F 3
category:  D 3


## Convert to Python dict

A `ParsedLabel` is a custom class and is not serializable by default. However, it is possible to convert it to a Python dict using the `to_dict` method:

In [ ]:
label = labels[0]
print(type(label))

<class 'kili.utils.labels.parsing.ParsedLabel'>


In [ ]:
label_as_dict = label.to_dict()
print(type(label_as_dict))

<class 'dict'>


## Transcription job

For a transcription job, the `.text` allows to access the label data:

In [ ]:
json_interface = {
    "jobs": {"TRANSCRIPTION_JOB": {"mlTask": "TRANSCRIPTION", "required": 1, "isChild": False}}
}

dict_label = {
    "jsonResponse": {"TRANSCRIPTION_JOB": {"text": "This is a transcription annotation..."}}
}

In [ ]:
label = ParsedLabel(dict_label, json_interface=json_interface, input_type="TEXT")

In [ ]:
print(label.jobs["TRANSCRIPTION_JOB"].text)

This is a transcription annotation...


## Object detection job

For object detection jobs, a parsed label has a `.annotations` attribute:

In [ ]:
json_interface = {
    "jobs": {
        "OBJECT_DETECTION_JOB": {
            "mlTask": "OBJECT_DETECTION",
            "tools": ["rectangle"],
            "required": 1,
            "isChild": False,
            "content": {"categories": {"A": {}, "B": {}}, "input": "radio"},
        }
    }
}

dict_label = {
    "jsonResponse": {
        "OBJECT_DETECTION_JOB": {
            "annotations": [
                {
                    "children": {},
                    "boundingPoly": [
                        {
                            "normalizedVertices": [
                                {"x": 0.54, "y": 0.52},
                                {"x": 0.54, "y": 0.33},
                                {"x": 0.70, "y": 0.33},
                                {"x": 0.70, "y": 0.52},
                            ]
                        }
                    ],
                    "categories": [{"name": "B"}],
                    "mid": "20230315142306286-25528",
                    "type": "rectangle",
                }
            ]
        }
    }
}

In [ ]:
label = ParsedLabel(dict_label, json_interface=json_interface, input_type="IMAGE")

In [ ]:
print(label.jobs["OBJECT_DETECTION_JOB"].annotations[0].category.name)

B


In [ ]:
print(label.jobs["OBJECT_DETECTION_JOB"].annotations[0].type)

rectangle


In [ ]:
print(label.jobs["OBJECT_DETECTION_JOB"].annotations[0].bounding_poly)

[{'normalizedVertices': [{'x': 0.54, 'y': 0.52}, {'x': 0.54, 'y': 0.33}, {'x': 0.7, 'y': 0.33}, {'x': 0.7, 'y': 0.52}]}]


In [ ]:
print(label.jobs["OBJECT_DETECTION_JOB"].annotations[0].bounding_poly[0].normalized_vertices)

[{'x': 0.54, 'y': 0.52}, {'x': 0.54, 'y': 0.33}, {'x': 0.7, 'y': 0.33}, {'x': 0.7, 'y': 0.52}]


The `.bounding_poly_annotations` attribute is equivalent to `.annotations`:

In [ ]:
print(
    label.jobs["OBJECT_DETECTION_JOB"].annotations
    == label.jobs["OBJECT_DETECTION_JOB"].bounding_poly_annotations
)

True


## Point detection job

The point coordinates of a point detection label is accessible through the `.point` attribute:

In [ ]:
json_interface = {
    "jobs": {
        "OBJECT_DETECTION_JOB": {
            "content": {
                "categories": {
                    "A": {"children": [], "color": "#472CED", "name": "A"},
                    "B": {"children": [], "name": "B", "color": "#5CE7B7"},
                },
                "input": "radio",
            },
            "instruction": "Class",
            "mlTask": "OBJECT_DETECTION",
            "required": 1,
            "tools": ["marker"],
            "isChild": False,
        }
    }
}

dict_label = {
    "jsonResponse": {
        "OBJECT_DETECTION_JOB": {
            "annotations": [
                {
                    "children": {},
                    "point": {"x": 0.10, "y": 0.20},
                    "categories": [{"name": "A"}],
                    "mid": "20230323113855529-11197",
                    "type": "marker",
                },
                {
                    "children": {},
                    "point": {"x": 0.30, "y": 0.40},
                    "categories": [{"name": "B"}],
                    "mid": "20230323113857016-51829",
                    "type": "marker",
                },
            ]
        }
    }
}

In [ ]:
label = ParsedLabel(dict_label, json_interface=json_interface, input_type="IMAGE")

In [ ]:
print(label.jobs["OBJECT_DETECTION_JOB"].annotations[1].type)

marker


In [ ]:
print(label.jobs["OBJECT_DETECTION_JOB"].annotations[1].point)

{'x': 0.3, 'y': 0.4}


In [ ]:
print(label.jobs["OBJECT_DETECTION_JOB"].annotations[1].category.name)

B


## Line detection job

A polyline parsed label has a `.polyline` attribute.

In [ ]:
json_interface = {
    "jobs": {
        "OBJECT_DETECTION_JOB": {
            "content": {
                "categories": {
                    "A": {"children": [], "color": "#472CED", "name": "A"},
                    "B": {"children": [], "name": "B", "color": "#5CE7B7"},
                },
                "input": "radio",
            },
            "instruction": "Job name",
            "mlTask": "OBJECT_DETECTION",
            "required": 1,
            "tools": ["polyline"],
            "isChild": False,
        }
    }
}

dict_label = {
    "jsonResponse": {
        "OBJECT_DETECTION_JOB": {
            "annotations": [
                {
                    "children": {},
                    "polyline": [{"x": 0.59, "y": 0.40}, {"x": 0.25, "y": 0.30}],
                    "categories": [{"name": "A"}],
                    "mid": "20230428163557647-23000",
                    "type": "polyline",
                },
                {
                    "children": {},
                    "polyline": [{"x": 0.70, "y": 0.50}, {"x": 0.40, "y": 0.70}],
                    "categories": [{"name": "B"}],
                    "mid": "20230428163606237-86143",
                    "type": "polyline",
                },
            ]
        }
    }
}

In [ ]:
label = ParsedLabel(dict_label, json_interface=json_interface, input_type="IMAGE")

In [ ]:
print(len(label.jobs["OBJECT_DETECTION_JOB"].annotations))

2


In [ ]:
print(label.jobs["OBJECT_DETECTION_JOB"].annotations[0].category.name)

'A'

In [ ]:
print(label.jobs["OBJECT_DETECTION_JOB"].annotations[0].polyline)

[{'x': 0.59, 'y': 0.4}, {'x': 0.25, 'y': 0.3}]


## Video job

A video label has an additional attribute `.frames` that returns the annotations for each frame.

In [ ]:
json_interface = {
    "jobs": {
        "FRAME_CLASSIF_JOB": {
            "content": {
                "categories": {
                    "OBJECT_A": {"children": [], "name": "Object A"},
                    "OBJECT_B": {"children": [], "name": "Object B"},
                },
                "input": "radio",
            },
            "instruction": "Categories",
            "isChild": False,
            "mlTask": "CLASSIFICATION",
            "models": {},
            "isVisible": False,
            "required": 1,
        }
    }
}

dict_label = {
    "jsonResponse": {
        "0": {},
        "1": {},
        "2": {},
        "3": {},
        "4": {},
        "5": {
            "FRAME_CLASSIF_JOB": {
                "categories": [{"confidence": 100, "name": "OBJECT_A"}],
                "isKeyFrame": True,
                "annotations": [],
            }
        },
        "6": {
            "FRAME_CLASSIF_JOB": {
                "categories": [{"confidence": 42, "name": "OBJECT_B"}],
                "isKeyFrame": False,
                "annotations": [],
            }
        },
        "7": {},
        "8": {},
    }
}

In [ ]:
label = ParsedLabel(dict_label, json_interface=json_interface, input_type="VIDEO")

In [ ]:
for i, frame_annotations in enumerate(label.jobs["FRAME_CLASSIF_JOB"].frames):
    print(f"Frame {i}: {frame_annotations}")

Frame 0: {}
Frame 1: {}
Frame 2: {}
Frame 3: {}
Frame 4: {}
Frame 5: {'isKeyFrame': True, 'categories': [{'name': 'OBJECT_A', 'confidence': 100}], 'annotations': []}
Frame 6: {'isKeyFrame': False, 'categories': [{'name': 'OBJECT_B', 'confidence': 42}], 'annotations': []}
Frame 7: {}
Frame 8: {}


In [ ]:
frame = label.jobs["FRAME_CLASSIF_JOB"].frames[5]

In [ ]:
print(frame.category.name)

OBJECT_A


The syntax is similar for object detection jobs on video:

In [ ]:
json_interface = {
    "jobs": {
        "JOB_0": {
            "content": {
                "categories": {
                    "OBJECT_A": {"children": [], "name": "Train", "color": "#733AFB"},
                    "OBJECT_B": {"children": [], "name": "Car", "color": "#3CD876"},
                },
                "input": "radio",
            },
            "instruction": "Track objects A and B",
            "isChild": False,
            "tools": ["rectangle"],
            "mlTask": "OBJECT_DETECTION",
            "models": {"tracking": {}},
            "isVisible": True,
            "required": 0,
        }
    }
}

dict_label = {
    "jsonResponse": {
        "0": {},
        "1": {
            "JOB_0": {
                "annotations": [
                    {
                        "children": {},
                        "boundingPoly": [
                            {
                                "normalizedVertices": [
                                    {"x": 0.30, "y": 0.63},
                                    {"x": 0.30, "y": 0.55},
                                    {"x": 0.36, "y": 0.55},
                                    {"x": 0.36, "y": 0.63},
                                ]
                            }
                        ],
                        "categories": [{"name": "OBJECT_B"}],
                        "mid": "20230407140827577-43802",
                        "type": "rectangle",
                        "isKeyFrame": True,
                    }
                ]
            }
        },
    }
}

In [ ]:
label = ParsedLabel(dict_label, json_interface=json_interface, input_type="VIDEO")

In [ ]:
print(label.jobs["JOB_0"].frames[1].annotations[0].category.name)

OBJECT_B


## Named entities recognition job

For NER jobs, the content of the job reponse is a list of annotations.

Those annotations can be accessed through the `.annotations` or `.entity_annotations` attributes.

In [ ]:
json_interface = {
    "jobs": {
        "NER_JOB": {
            "mlTask": "NAMED_ENTITIES_RECOGNITION",
            "required": 1,
            "isChild": False,
            "content": {
                "categories": {"ORG": {}, "PERSON": {}},
                "input": "radio",
            },
        }
    }
}
dict_label = {
    "jsonResponse": {
        "NER_JOB": {
            "annotations": [
                {
                    "categories": [{"name": "ORG", "confidence": 42}],
                    "beginOffset": 21,
                    "content": "this is the text for Kili",
                    "mid": "mid_a",
                },
                {
                    "categories": [{"name": "PERSON", "confidence": 100}],
                    "beginOffset": 8,
                    "content": "this is Toto's text",
                    "mid": "mid_b",
                },
            ]
        }
    }
}

In [ ]:
label = ParsedLabel(dict_label, json_interface=json_interface, input_type="TEXT")

In [ ]:
print("Number of annotations in this label: ", len(label.jobs["NER_JOB"].annotations))

Number of annotations in this label:  2


In [ ]:
print(label.jobs["NER_JOB"].annotations[0].category)

{'name': 'ORG', 'confidence': 42}


In [ ]:
print(label.jobs["NER_JOB"].annotations[0].begin_offset)

21


In [ ]:
print(label.jobs["NER_JOB"].annotations[0].content)

this is the text for Kili


In [ ]:
print(label.jobs["NER_JOB"].annotations[0].mid)

mid_a


It is also possible to iterate over the annotations of this label:

In [ ]:
for annotation in label.jobs["NER_JOB"].annotations:
    print(annotation)

{'categories': [{'name': 'ORG', 'confidence': 42}], 'beginOffset': 21, 'content': 'this is the text for Kili', 'mid': 'mid_a'}
{'categories': [{'name': 'PERSON', 'confidence': 100}], 'beginOffset': 8, 'content': "this is Toto's text", 'mid': 'mid_b'}


## Named entities recognition in PDF job

## Relation job

### Named entities relation job

### Object detection relation job

## Pose estimation job

## Children jobs